In [1]:
# pip install google-api-python-client youtube-transcript-api openai
# pip install sentencepiece
# openai --version # 1.34.0

# pip install transformers
# pip install torch
# pip install google-api-python-client
# pip install youtube-transcript-api


In [2]:
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import os

# Replace with your API keys and channel ID
API_KEY = 'AIzaSyDeGQSEB_xxxxxxxxxxxxxxxxx'
CHANNEL_ID = 'UCXuqSxxxxxxxxxxxxxxxxx'
OPENAI_API_KEY = 'sk-proj-YQ1mbN9Hxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx' # not needed as local model

/home/harsh/anaconda3/envs/llm_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Directory to save and load models
model_dir = "./model"
os.makedirs(model_dir, exist_ok=True)  # Create the model directory if it doesn't exist

# Initialize the T5 model and tokenizer
model_name = 't5-small'  # you can use 't5-base' for better performance

# Save the model and tokenizer
def save_model(model_name, model_dir):
    """
    Save the T5 model and tokenizer to the specified directory.

    Args:
        model_name (str): The name of the model to download and save.
        model_dir (str): The directory where the model and tokenizer will be saved.
    """
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model.save_pretrained(model_dir)
    tokenizer.save_pretrained(model_dir)


In [4]:
# Load the model and tokenizer
def load_model(model_dir):
    """
    Load the T5 model and tokenizer from the specified directory.

    Args:
        model_dir (str): The directory from where the model and tokenizer will be loaded.

    Returns:
        model: The loaded T5 model.
        tokenizer: The loaded T5 tokenizer.
    """
    model = T5ForConditionalGeneration.from_pretrained(model_dir).to(device)
    tokenizer = T5Tokenizer.from_pretrained(model_dir)
    return model, tokenizer


In [5]:
# Check if the model already exists in the directory; if not, download and save it
if not os.path.exists(os.path.join(model_dir, 'config.json')):
    save_model(model_name, model_dir)

# Set the device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the model and tokenizer
model, tokenizer = load_model(model_dir)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
print(device)

cuda


In [7]:
def get_latest_video(api_key, channel_id):
    """
    Retrieve the latest video from a specified YouTube channel.

    Args:
        api_key (str): YouTube API key.
        channel_id (str): YouTube channel ID.

    Returns:
        video_id (str): The ID of the latest video.
        video_title (str): The title of the latest video.
    """
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # Create a request to get the latest video from the channel
    request = youtube.search().list(
        part='snippet',
        channelId=channel_id,
        order='date',
        maxResults=1
    )
    response = request.execute()
    
    # Check if there are any videos in the response
    if 'items' in response:
        latest_video = response['items'][0]
        video_id = latest_video['id']['videoId']
        video_title = latest_video['snippet']['title']
        return video_id, video_title
    else:
        return None, None


In [8]:
def get_transcript(video_id):
    """
    Retrieve the transcript of a specified YouTube video.

    Args:
        video_id (str): The ID of the YouTube video.

    Returns:
        transcript_text (str): The full transcript text of the video.
    """
    try:
        # Get the transcript of the video
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Join all the transcript segments into a single string
        transcript_text = ' '.join([item['text'] for item in transcript])
        return transcript_text
    except Exception as e:
        print(f"Error retrieving transcript: {e}")
        return None

In [9]:
def summarize_text(text):
    """
    Summarize the provided text using the T5 model.

    Args:
        text (str): The text to be summarized.

    Returns:
        summary (str): The summarized text.
    """
    # Encode the input text and move it to the appropriate device
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True).to(device)
    # Generate the summary using the T5 model
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    # Decode the generated summary and return it
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [10]:
def main():
    # Get the latest video from the specified channel
    video_id, video_title = get_latest_video(API_KEY, CHANNEL_ID)
    if video_id:
        print(f"Latest video ID: {video_id}, Title: {video_title}")
        # Get the transcript of the latest video
        transcript = get_transcript(video_id)
        if transcript:
            print("Transcript retrieved successfully.")
            # Summarize the transcript
            summary = summarize_text(transcript)
            print("Summary:")
            print(summary)
        else:
            print("Failed to retrieve transcript.")
    else:
        print("No videos found.")

if __name__ == "__main__":
    main()

Latest video ID: uxsTpMXzEU0, Title: Hey Siri, I’m Giving You A Second Chance
Transcript retrieved successfully.
Summary:
new AI enhanced functions are now built into all of apple's operating systems that allow you to quickly generate smart replies proofread or rewrite messages and summarize content which is probably the most useful tool for me though is reduce interruptions. how is all of this working well Apple claims that their machine learning algorithms primarily run on device as they always have but because of the increasing complexity of these tasks.
